Data is extracted from chembl_24.db (from [ChEMBL database](https://www.ebi.ac.uk/chembl/downloads)). We then parse the data and save data into pickle file for use in deep learning of the structures.

Import packages

In [1]:
import numpy as np
import pandas as pd
import os
import sqlite3
import pickle
import rdkit.Chem as chem

from sklearn.utils import shuffle

Connect database using sqlite3 package

In [5]:
db = sqlite3.connect('chembl_24.db')
c = db.cursor()

Import doc_id from chembl_24.db, then use doc_id to extract molregno (unique internal Chembl compound identifier)

In [6]:
categories = ['%toxin%', '%fungicidal%', '%nematicidal%', '%herbicidal%', '%insecticidal%']

In [7]:
molregno = dict.fromkeys(categories, None)
for cat in categories:
    # extract doc_id from assays that contain agrochemical and non-agrochemical keywords
    doc_id = c.execute("SELECT doc_id FROM assays where description like '%s'" %cat).fetchall()
    doc_id = [i[0] for i in doc_id]
    
    # extract unique compound identifier from doc_id
    molregno[cat] = c.execute("SELECT molregno FROM compound_records WHERE doc_id IN " + str(tuple(doc_id))).fetchall()
    molregno[cat] = [i[0] for i in molregno[cat]]
    print ("%s" %cat[1:-1], ":", len(molregno[cat]))

toxin : 541388
fungicidal : 4678
nematicidal : 555
herbicidal : 3715
insecticidal : 5987


Dispose data that overlaps 

In [8]:
%%time
for i, cat_a in enumerate(categories):
    for j, cat_b in enumerate(categories):
        if (i < j):
            intersection = list(set(molregno[cat_a])&set(molregno[cat_b]))
            molregno[cat_a] = [x for x in molregno[cat_a] if x not in intersection]

CPU times: user 7.06 s, sys: 122 ms, total: 7.18 s
Wall time: 7.37 s


Then, we get canonical smiles string and compound properties from molregno compound identifier. We check to make sure the smiles strings and properties are mapped exactly.

- mw_freebase = Molecular weight of parent compound
- alogp = Calculated ALogP
- hba = number of hydrogen bond acceptors
- hbd = number of hydrogen bond donors
- psa = polar surface area
- rtb = number of rotatable bonds
- acd_logp = calculated octanol/water partition coefficient using ACDlabs v12.01
- acd_logd = calculated octanol/water distribution coefficient at pH 7.4 using ACDlabs v12.01
- full_mwt = molecular weight of the full compound including any salts
- aromatic_rings = number of aromatic rings
- heavy_atoms = number of heavy (non-hydrogen) atoms
- qed_weighted = weighted quantitative estimate of drug likeness
- mw_monoisotopic = monoisotopic parent molecular weight
- hba_lipinski = number of hydrogen bond acceptros calculated according to the Lipinski's original rules (i.e. N + O count)
- hbd_lipinski = number of hydrogen bond donors calculated according to the Lipinski's original rules (i.e., NH + OH count)

In [9]:
smiles_string = dict.fromkeys(categories, None)
mw_freebase_dict = dict.fromkeys(categories, None)
alogp_dict = dict.fromkeys(categories, None)
hba_dict = dict.fromkeys(categories, None)
hbd_dict = dict.fromkeys(categories, None)
psa_dict = dict.fromkeys(categories, None)
rtb_dict = dict.fromkeys(categories, None)
acd_logp_dict = dict.fromkeys(categories, None)
acd_logd_dict = dict.fromkeys(categories, None)
full_mwt_dict = dict.fromkeys(categories, None)
aromatic_rings_dict = dict.fromkeys(categories, None)
heavy_atoms_dict = dict.fromkeys(categories, None)
qed_weighted_dict = dict.fromkeys(categories, None)
mw_monoisotopic_dict = dict.fromkeys(categories, None)
hba_lipinski_dict = dict.fromkeys(categories, None)
hbd_lipinski_dict = dict.fromkeys(categories, None)

In [10]:
for cat in molregno:
    smiles_string[cat], mw_freebase_dict[cat], alogp_dict[cat], hba_dict[cat], hbd_dict[cat], psa_dict[cat] = [], [], [], [], [], []
    rtb_dict[cat], acd_logp_dict[cat], acd_logd_dict[cat], full_mwt_dict[cat], aromatic_rings_dict[cat], heavy_atoms_dict[cat] = [], [], [], [], [], []
    qed_weighted_dict[cat], mw_monoisotopic_dict[cat], hba_lipinski_dict[cat], hbd_lipinski_dict[cat] = [], [], [], []
    for num in molregno[cat]:
        smile = c.execute("SELECT canonical_smiles FROM compound_structures WHERE molregno = " + str(num)).fetchall()
        properties = c.execute("SELECT * FROM compound_properties WHERE molregno = " + str(num)).fetchall()
        if not smile or not properties:
            molregno[cat].remove(num)
        else:
            properties = properties[0]
            smiles_string[cat].append(smile[0])
            
            # assign properties to corresponding dictionaries
            mw_freebase_dict[cat].append(properties[1])
            alogp_dict[cat].append(properties[2])
            hba_dict[cat].append(properties[3])
            hbd_dict[cat].append(properties[4])
            psa_dict[cat].append(properties[5])
            rtb_dict[cat].append(properties[6])
            acd_logp_dict[cat].append(properties[11])
            acd_logd_dict[cat].append(properties[12])
            full_mwt_dict[cat].append(properties[14])
            aromatic_rings_dict[cat].append(properties[15])
            heavy_atoms_dict[cat].append(properties[16])
            qed_weighted_dict[cat].append(properties[17])
            mw_monoisotopic_dict[cat].append(properties[18])
            hba_lipinski_dict[cat].append(properties[20])
            hbd_lipinski_dict[cat].append(properties[21])

Convert smiles string and the properties into a long list and then create another list containing their corresponding categorical name

In [11]:
canonical_smiles, label = [], []
mw_freebase, alogp, hba, hbd, psa, rtb, acd_logp, acd_logd, full_mwt, aromatic_rings, heavy_atoms, qed_weighted, mw_monoisotopic, hba_lipinski, hbd_lipinski = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],
for cat in smiles_string:
    canonical_smiles += smiles_string[cat]
    label += [cat[1:-1]]*len(smiles_string[cat])
    
    mw_freebase += mw_freebase_dict[cat]
    alogp += alogp_dict[cat]
    hba += hba_dict[cat]
    hbd += hbd_dict[cat]
    psa += psa_dict[cat]
    rtb += rtb_dict[cat]
    acd_logp += acd_logp_dict[cat]
    acd_logd += acd_logd_dict[cat]
    full_mwt += full_mwt_dict[cat]
    aromatic_rings += aromatic_rings_dict[cat]
    heavy_atoms += heavy_atoms_dict[cat]
    qed_weighted += qed_weighted_dict[cat]
    mw_monoisotopic += mw_monoisotopic_dict[cat]
    hba_lipinski += hba_lipinski_dict[cat]
    hbd_lipinski += hbd_lipinski_dict[cat]
    

Stack the two lists together

In [12]:
data = np.column_stack((canonical_smiles, label, mw_freebase, alogp, hba, hbd, psa, rtb, acd_logp, acd_logd, full_mwt, aromatic_rings, heavy_atoms, qed_weighted, mw_monoisotopic, hba_lipinski, hbd_lipinski))

Convert data into pandas dataframe

In [13]:
data = pd.DataFrame(data, columns=['smiles', 'category', 'mw_freebase', 'alogp', 'hba', 'hbd', 'psa', 'rtb', 'acd_logp', 'acd_logd', 'full_mwt', 'aromatic_rings', 'heavy_atoms', 'qed_weighted', 'mw_monoisotopic', 'hba_lipinski', 'hbd_lipinski'])

Add a column containing RDKit Molecule class

In [14]:
%%time
data['mol'] = data['smiles'].apply(chem.MolFromSmiles)

CPU times: user 3min 4s, sys: 4.98 s, total: 3min 9s
Wall time: 3min 9s


Add another column that differentiates non-agrochemical and agrochemical category

In [15]:
def class_label(x):
    if x == 'toxin':
        return 0
    elif x == 'herbicidal':
        return 1
    elif x == 'insecticidal':
        return 2
    elif x == 'nematicidal':
        return 3
    elif x == 'fungicidal':
        return 4

In [16]:
%%time
data['agrochemical'] = data['category'].apply(class_label)

CPU times: user 210 ms, sys: 14.3 ms, total: 225 ms
Wall time: 226 ms


Remove null values

In [17]:
data.dropna(axis=0, inplace=True)

Reset index

In [18]:
data.reset_index(drop=True, inplace=True)

Count number of compounds in agrochemical and non-agrochemical category

In [21]:
data['agrochemical'].value_counts()

0    529239
2      5834
4      3822
1      3526
3       405
Name: agrochemical, dtype: int64

In [22]:
data

,smiles,category,mw_freebase,alogp,hba,hbd,psa,rtb,acd_logp,acd_logd,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,hba_lipinski,hbd_lipinski,mol,agrochemical
0,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,840.97,-0.94,15,8,279.95,7,0.74,0.7,840.97,0,59,0.11,840.459,21,8,<rdkit.Chem.rdchem.Mol object at 0x135db82b0>,0
1,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,839,-0.7,14,7,259.72,7,1.18,1.15,839,0,59,0.12,838.48,20,7,<rdkit.Chem.rdchem.Mol object at 0x135db81c0>,0
2,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,782.89,-2.06,14,8,268.51,6,-0.15,-0.18,782.89,0,55,0.11,782.417,20,8,<rdkit.Chem.rdchem.Mol object at 0x135db8350>,0
3,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,839.98,-0.05,14,7,267.92,7,1.94,1.88,839.98,0,59,0.13,839.464,20,7,<rdkit.Chem.rdchem.Mol object at 0x135db8260>,0
4,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,885.03,-1.03,15,9,305.81,11,1.75,1.67,885.03,0,62,0.07,884.486,22,9,<rdkit.Chem.rdchem.Mol object at 0x135db83f0>,0
5,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,869.03,-0.59,15,6,257.95,9,1.13,1.13,869.03,0,61,0.16,868.491,21,6,<rdkit.Chem.rdchem.Mol object at 0x135db8300>,0
6,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,808.97,-1.14,13,8,257.07,7,1.4,1.4,808.97,0,57,0.14,808.47,19,8,<rdkit.Chem.rdchem.Mol object at 0x135db8490>,0
7,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,939.12,0.02,16,7,286.02,9,1.48,1.44,939.12,0,66,0.12,938.532,22,7,<rdkit.Chem.rdchem.Mol object at 0x135db83a0>,0
8,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,867.01,-0.78,15,7,276.79,7,1.32,1.28,867.01,0,61,0.12,866.475,21,7,<rdkit.Chem.rdchem.Mol object at 0x135db8530>,0
9,CC[C@H](C)C[C@H]1CC[C@@](O)(O[C@@H]1C)[C@](C)(...,toxin,931.1,0.81,15,7,268.95,10,2.47,2.43,931.1,1,66,0.13,930.506,21,7,<rdkit.Chem.rdchem.Mol object at 0x135db8440>,0


### Save data as pickle file labeled dataset2 (all data)

In [23]:
data.to_pickle("./multiclass_classification/data/dataset2.pkl")

### Save another dataset: dataset1 (balanced dataset with approximately equal proportion of agro and non-agrochemicals) 

In [25]:
agrochemicals = data.loc[data['agrochemical'] != 0]
nonagrochemicals = data.loc[data['agrochemical'] == 0]

In [28]:
nonagrochemicals = shuffle(nonagrochemicals)
nonagrochemicals = nonagrochemicals[:15000]

In [29]:
dataset1 = pd.concat([agrochemicals, nonagrochemicals], axis=0)

In [30]:
dataset1

,smiles,category,mw_freebase,alogp,hba,hbd,psa,rtb,acd_logp,acd_logd,full_mwt,aromatic_rings,heavy_atoms,qed_weighted,mw_monoisotopic,hba_lipinski,hbd_lipinski,mol,agrochemical
529239,CC(C)c1ccnc(c1)c2cc(ccn2)C(C)C,fungicidal,240.35,4.39,2,0,25.78,3,4.68,4.68,240.35,2,18,0.79,240.163,2,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c080>,4
529240,COc1ccccc1c2ccnc(c2)c3cc(C)ccn3,fungicidal,276.34,4.13,3,0,35.01,3,3.34,3.34,276.34,3,21,0.72,276.126,3,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c0d0>,4
529241,Cc1ccnc(c1)c2cc(ccn2)c3ccccc3,fungicidal,246.31,4.12,2,0,25.78,2,3.64,3.64,246.31,3,19,0.68,246.116,2,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c120>,4
529242,COc1ccccc1c2ccnc(c2)c3ccccn3,fungicidal,262.31,3.82,3,0,35.01,3,2.84,2.84,262.31,3,20,0.72,262.111,3,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c170>,4
529243,[O-][N+](=O)c1ccccc1c2ccnc(c2)c3ccccn3,fungicidal,277.28,3.72,4,0,68.92,3,2.26,2.26,277.28,3,21,0.54,277.085,5,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c1c0>,4
529244,c1ccc(cc1)c2ccnc(c2)c3ccccn3,fungicidal,232.29,3.81,2,0,25.78,2,3.14,3.14,232.29,3,18,0.67,232.1,2,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c210>,4
529245,Cc1ccnc(c1)c2ccc(cn2)c3ccccc3,fungicidal,246.31,4.12,2,0,25.78,2,3.66,3.66,246.31,3,19,0.68,246.116,2,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c260>,4
529246,[O-][N+](=O)c1ccc(cc1)c2ccc(nc2)c3ccccn3,fungicidal,277.28,3.72,4,0,68.92,3,2.75,2.75,277.28,3,21,0.54,277.085,5,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c2b0>,4
529247,COc1ccc(cc1)c2ccc(nc2)c3ccccn3,fungicidal,262.31,3.82,3,0,35.01,3,2.98,2.98,262.31,3,20,0.72,262.111,3,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c300>,4
529248,Cc1ccc(cc1)c2ccc(nc2)c3ccccn3,fungicidal,246.31,4.12,2,0,25.78,2,3.71,3.71,246.31,3,19,0.68,246.116,2,0,<rdkit.Chem.rdchem.Mol object at 0x13bd5c350>,4


In [31]:
dataset1.to_pickle("./multiclass_classification/data/dataset1.pkl")